# Neural network demo with PyTorch

# Convolutional Neural Network tutorial

This is an example of how neural networks can be used to categorize images.  The test images are the MNIST collection of handwritten digits, and we will train a network to recognized hand-written digits.  Other sets of data are available.

Installation info:
* numpy must be 1.X to work with torch.  ``python -m pip install 'numpy<2'``

### Import packages and dataset


In [1]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets
from torchvision.transforms import v2 as transforms
import matplotlib.pyplot as plt


Tell PyTorch to run on the cpu.  Set up a random number generator.

In [ ]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

if use_cuda:
    device = torch.device('cuda')
    print('Running on Nvidia GPU (cuda)')
elif use_mps:
    device = torch.device('mps')
    print('Running on Apple Silicon GPU device (mps)')
else:
    device = torch.device('cpu')
    print('Running on CPU')
    
rng = np.random.default_rng()

### Select and look at the dataset

We start with the standard **MNIST** dataset of handwritten digits.

Harder data sets are **CIFAR10** and **CIFAR100**, tiny color images with 10 or 100 categories.  

An interesting data set is **Imagenette**, it exists in a 160px and 320px version.  Even the 160px version is too hard for this simple network but may work for more complicated neural network architectures.

Complication: the images are of different size, and our network requires that they are the same.  A transformation can be applied to scale all images to same size.  This dataset also takes different parameters to split into training and dataset, see the documentation at https://pytorch.org/vision/stable/datasets.html

Data scientists always start by visualizing their data - avoid surprises later.

In [ ]:
# Transformation for the MNIST and CIFAR datasets: Convert to tensor of float32
transform=transforms.Compose([
        transforms.ToImage(),   # Converts PIL images to tensors
        transforms.ToDtype(torch.float32, scale=True),
        ])

# Transformation for the Imagenette dataset: Convert and rescale to 160 x 160
transform_imagenette=transforms.Compose([
        transforms.ToImage(),   # Converts PIL images to tensors
        transforms.Resize((160,160)),
        transforms.ToDtype(torch.float32, scale=True),
        ])

#transform = transforms.ToTensor()

dataset_train = datasets.MNIST('data', train=True, download=True, transform=transform)
dataset_test = datasets.MNIST('data', train=False, transform=transform)
#dataset_train = datasets.CIFAR100('data', train=True, download=True, transform=transform)
#dataset_test = datasets.CIFAR100('data', train=False, transform=transform)
#dataset_train = datasets.Imagenette('data2', split='train', size='160px', download=True, transform=transform_imagenette)
#dataset_test = datasets.Imagenette('data2', split='val', size='160px', transform=transform_imagenette)

ntrain = len(dataset_train)
ntest = len(dataset_test)
imgsize = dataset_test[0][0].shape
print(imgsize)
num_categories = len(dataset_train.classes)
print("Training set: {} images of shape {}".format(ntrain, list(imgsize)))
print("Pixels in image", np.prod(imgsize))
print("Number of categories:", num_categories)
print("Categories:", dataset_train.classes)
print("Training set identifies as")
print(dataset_train)
print("Test set identifies as")
print(dataset_test)

In [ ]:
# Sanity check of data types
print(dataset_test[0][0].shape, dataset_test[0][0].dtype)
print(dataset_test[0][0].min(), dataset_test[0][0].max())


In [ ]:
nplot = 10
rnd = rng.integers(len(dataset_test), size=(nplot,))
fig, axes = plt.subplots(1, nplot, figsize=(12,12/nplot))
for i in range(nplot):
    image, label = dataset_test[rnd[i]]
    print(image.shape)
    # Shape (1, X, Y) if BW, (3, X, Y) if color
    if image.shape[0] == 1:
        image = image[0]  # New shape (X, Y)
    elif image.shape[0] in (3, 4):
        image = torch.moveaxis(image, 0, -1)    # New shape (X, Y, 3)
    axes[i].imshow(image, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(label)


### Define a function creating the neural network
Make multiple functions to be able to experiment with multiple architectures.

In [6]:
def make_minimalnet(categories, imshape, channels=10, hiddenneurons=200):
    inchannels = imshape[0]
    imsize = imshape[1] * imshape[2]
    return nn.Sequential(
        nn.Conv2d(inchannels, channels, kernel_size=5, padding='same'),
        nn.Sigmoid(),
        nn.MaxPool2d(2),
        nn.Conv2d(channels, channels, kernel_size=5, padding='same'),
        nn.Sigmoid(),
        nn.Flatten(1),
        nn.Linear(channels * imsize // 4, hiddenneurons),
        nn.Sigmoid(),
        nn.Linear(hiddenneurons, categories),
        nn.LogSoftmax(dim=1)
    )


### Define training and test functions

This is more or less boilerplate code, but you may want to modify it a little for other applications.

In [7]:
lossfunc = F.nll_loss   # Pick loss function for both functions below.

def train(model, device, train_loader, optimizer, epoch, verbose=False):
    """Train model for a single epoch.
    
    Parameters:
    model:         PyTorch model to train.
    device:        Device used for training (i.e. cpu, gpu, ...)
    train_loader:  Data loader
    optimizer:     The optimizer used in the training.
    epoch:         Epoch number
    """
    model.train()  # Put model in training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = lossfunc(output, target)
        loss.backward()
        optimizer.step()
        # Print progress
        if verbose:
            ndone = batch_idx * len(data)
            ntot = len(train_loader.dataset)
            percent = 100 * ndone / ntot
            print(f'Train epoch: {epoch} [{ndone}/{ntot} ({percent:.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader, epoch):
    """Evaluate a model.
    
    It does not return anything, but prints the test performance.

    Parameters:
    model:       PyTorch model
    device:      Device used for training / evaluating.
    test_loader: Data loader.
    """
    model.eval()  # Put model in evaluation mode.
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += lossfunc(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    ndata = len(test_loader.dataset)
    percent = 100. * correct / ndata
    print(f'Test set, epoch {epoch}: Average loss: {test_loss:.4f}, Accuracy: {correct}/{ndata} ({percent:.0f}%)\n')


### Do the training.

Define
* Data loaders
* model
* optimizer  (adjusts weights in model)
* scheduler  (adjusts learning rate in optimizer)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=64)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1000)

model = make_minimalnet(num_categories, imgsize).to(device)
#optimizer = optim.Adadelta(model.parameters(), lr=0.1)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Decay learning rate exponentially
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

print(model)

In [ ]:
nepochs = 10
for epoch in range(nepochs):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader, epoch)
    scheduler.step()

### Show some of the failures

We run the network on the test data again, this time keeping the results and looking at them.

We need to run the model on the input.  The variable ``dataset_test`` is a sequence of tuples of (image, label).  We need to use a DataLoader to get it converted to the right types and splitting it into images and labels.  We are lazy here and use ``test_loader`` and just get the first batch from it, so we only look at the first 1000 datapoints, which should be fine.

In [ ]:
images, labels = next(iter(test_loader))
print(images.shape, images.dtype, labels.shape, labels.dtype)
with torch.no_grad():   # Don't waste memory and time on the gradients!
    predictions = model(images.to(device))
print(predictions.shape, predictions.dtype)

# Convert to numpy arrays for further processing.
predictions = predictions.cpu().detach().numpy()
labels = labels.cpu().detach().numpy()
print(predictions.shape, predictions.dtype)


The output is a matrix of *logarithms* of probabilities.  Each row contains the probabilities that the image belongs to each of the ten classes.  Let us look at the first five.

Let us also take the exponentials to get the actual probabilities, and check that the sums are 1.0

This kind of sanity checks are important when working with complex libraries like PyTorch, at least until you are very familiar with what they are doing.

In [ ]:
with np.printoptions(precision=4, suppress=True):
    print(predictions[:5])
    print(np.exp(predictions[:5]))
    print(np.exp(predictions[:5]).sum(axis=1))

Convert this into predictions: find the most probable class for each image.

In [ ]:
predicted_class = np.argmax(predictions, axis=1)
print(predicted_class.shape)

n_error = (labels != predicted_class).sum()
print(f"There are {n_error} errors out of {len(predicted_class)}.")

Find the failures, show the first five.

In [ ]:
failures = np.argsort(labels != predicted_class)[-n_error:]

nshow = 5
fig, axes = plt.subplots(1, nshow, figsize=(12,12/nshow))

classes = dataset_test.classes
for i, f in enumerate(failures[:nshow]):
    img = images[f]   # Shape (1, X, Y) if BW, (3, X, Y) if color
    if img.shape[0] == 1:
        img = img[0]  # New shape (X, Y)
    elif img.shape[0] in (3, 4):
        img = torch.moveaxis(img, 0, -1)    # New shape (X, Y, 3)
    axes[i].imshow(img,cmap='gray')
    axes[i].axis('off')
    prd = predicted_class[f]
    prob = np.exp(predictions[f, prd])
    title = f'''{f}
Predicted: 
{prd} "{classes[prd]}"
Correct class: 
{labels[f]} "{classes[labels[f]]}"
Probability {100*prob:.1f}%'''
    axes[i].set_title(title)


In [ ]:
with np.printoptions(precision=4, suppress=True):
    print(np.exp(predictions[839]))

In [ ]:
dataset_test.classes